### Project goal:

* in this project we will analyse a dataset from a bank and prepare a review of what are the main points that can raise the risk of a customer defaulting on aloan. 
* we will focus on 4 different points:
1. Children
2. Family status( Marital Status)
3. Income level
4. purpose of the loan.

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer() 
from collections import Counter

In [2]:
credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')
credit_scoring.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [3]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# General Info and Notes:
1. there are missing values in 'days_employed' and 'total_income' and looks like the same amount, should check if in the same rows.
2. Days of employment 'days_employed' type should be changed
3. name of 'dob_years' colmun should be changed to 'age'

# Changing column name

In [4]:
credit_scoring = credit_scoring.rename(columns={'dob_years':'age'})  # rename the column
credit_scoring.columns

Index(['children', 'days_employed', 'age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

### Afterthoughts

* the name of the column 'dob_years' was misleading so it was changed to 'age'

# Filling Missing Values:

In [5]:
credit_scoring.isna().sum()   # check for missing values

children               0
days_employed       2174
age                    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
len(credit_scoring.loc[(credit_scoring['days_employed'].isna()) & (credit_scoring['total_income'].isna())])

2174

In [7]:
credit_scoring['days_employed'].head(15)

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
Name: days_employed, dtype: float64

In [8]:
count_rows = 0
successful_rows = 0
erroneous_rows = 0
unconverted_values= []


for value in credit_scoring['days_employed']:
    count_rows += 1
    try:
        value = int(value)
        successful_rows += 1
    except:
        erroneous_rows += 1
        unconverted_values.append(value)
        
        
print('Total rows', count_rows)        
print('Total successfully converted rows', successful_rows)        
print('Total rows with errors', erroneous_rows) 

unconverted_values[:10]

Total rows 21525
Total successfully converted rows 19351
Total rows with errors 2174


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

In [9]:
credit_scoring['days_employed'] = credit_scoring['days_employed'].fillna(credit_scoring['days_employed'].median())
credit_scoring.describe()

,children,days_employed,age,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [10]:
credit_scoring['total_income'].mean()

26787.56835465867

In [11]:
credit_scoring['total_income'].median()

23202.87

In [12]:
credit_scoring_incomefilled = credit_scoring.copy()
credit_scoring_incomefilled['total_income'] = credit_scoring['total_income'].fillna(credit_scoring['total_income'].mean())

In [13]:
credit_scoring_incomefilled.isna().sum() 

children            0
days_employed       0
age                 0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Afterthoughts

1. the dataset have missing values in both the 'days_employed' and 'total_income' and in the same rows for both columns and this possibly pints out that the data were in the begining as 2 or 3 tables(files) where one of them have less Information than the others and after merging the data that where missing was filled with (NaN). or it could be easily because of a corrupted file but I think that the first point is more likely.


2. while trying to change the type of days in 'days_employed' to int we encountered an error and with using the try-except method I found that the number of values with errors is the same as the missing values so that means all the other values can be converted. therefore we need to fill the missing values and then try to change the type later.


3. fill the missing values in 'days_employed' with the median
* **Note:the values here have negative values and very huge values(unrealistic) and I couldn't realy determine where the error was and why they are like this so for now as they days of employment doesn't have a big impact on the analysis i chose to fill it with median.**

4. fill the missing values in 'total_income' with the mean 
* **Note: the mean is heigher than the median in the total income which indicates a high outlier but as both of them will fall in same category of income level so they are basically the same so fo now I will use the mean.**


* **Note:I thought about filling the values with the mean based on the 'income_type' and tried to do so in a side notebook but as I found that it did not make an impact on the analysis and the results so i chose to fill it with the overall mean for simplicity.**


# Data type replacement and in depth preprocessing

## Children:

In [14]:
credit_scoring_incomefilled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [15]:
credit_scoring_incomefilled['children'].unique()  # weird that there is negative and a huge number of children

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [16]:
credit_scoring_incomefilled['children'].value_counts()  

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [17]:
credit_scoring_incomefilled['children'] = credit_scoring_incomefilled['children'].abs()
credit_scoring_incomefilled['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [18]:
credit_scoring_incomefilled.loc[credit_scoring_incomefilled['children'] == 20] = 2

In [19]:
credit_scoring_incomefilled['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Afterthoughts: 

1. there were '-1' and '20' as number of children.


2. as for the '-1' and because it is impossible to have a negative amount of children it I considered it as a typo in input and took its absolute value.


3. as for the '20' and considering the huge gap between the second top of number of children '5' and amount of people having 20 kids I found it not logical so I also considered it as typo and that it was meant to be 2


## Days of Employment

In [20]:
credit_scoring_incomefilled['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [21]:
# these are days so the numbers shouldn't be floats but integers
credit_scoring_incomefilled['days_employed'] = credit_scoring_incomefilled['days_employed'].astype('int')
credit_scoring_incomefilled['days_employed']

0         -8437
1         -4024
2         -5623
3         -4124
4        340266
          ...  
21520     -4529
21521    343937
21522     -2113
21523     -3112
21524     -1984
Name: days_employed, Length: 21525, dtype: int64

In [22]:
# days can't be negative
credit_scoring_incomefilled['days_employed'] = credit_scoring_incomefilled['days_employed'].abs()
credit_scoring_incomefilled['days_employed']

0          8437
1          4024
2          5623
3          4124
4        340266
          ...  
21520      4529
21521    343937
21522      2113
21523      3112
21524      1984
Name: days_employed, Length: 21525, dtype: int64

### Afterthoughts: 

1. as these are days so they should be positive integer values so I took the absolute values and changed the type from 'float' to 'int'. 
**Note: as I think these numbers are not very important in the overall results so for now I will stop here without any further indepth analysis.**

## Age:

In [23]:
age = credit_scoring_incomefilled['age'].unique()
age

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69,  2, 19, 72, 70, 74, 75])

In [24]:
age.sort()
age

array([ 0,  2, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75])

In [25]:
# there are weird values in the age column (0 - 2)
ages = credit_scoring_incomefilled['age'].value_counts()
ages

35    615
40    605
41    605
34    600
38    597
42    594
33    579
39    572
31    558
36    553
44    545
29    543
30    537
48    537
37    533
43    511
50    511
32    508
49    505
28    503
45    494
27    491
52    483
56    482
47    480
54    478
46    472
58    461
57    459
53    458
51    447
55    442
59    442
26    407
60    376
25    356
61    354
62    351
63    269
64    264
24    263
23    253
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     84
2      76
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: age, dtype: int64

In [26]:
#lets find how many weird values there are
ages.sort_index()

0     100
2      76
19     14
20     51
21    110
22    183
23    253
24    263
25    356
26    407
27    491
28    503
29    543
30    537
31    558
32    508
33    579
34    600
35    615
36    553
37    533
38    597
39    572
40    605
41    605
42    594
43    511
44    545
45    494
46    472
47    480
48    537
49    505
50    511
51    447
52    483
53    458
54    478
55    442
56    482
57    459
58    461
59    442
60    376
61    354
62    351
63    269
64    264
65    194
66    183
67    167
68     99
69     84
70     65
71     58
72     33
73      8
74      6
75      1
Name: age, dtype: int64

In [27]:
# percentage of the rows with 0 and 2 in the age to the whole dataframe
age_low = len(credit_scoring_incomefilled.loc[credit_scoring_incomefilled['age']< 18])
age_all = len(credit_scoring_incomefilled['age'])
age_low_per = age_low/age_all
age_low_per

0.008176538908246225

In [28]:
credit_scoring_incomefilled = credit_scoring_incomefilled.loc[credit_scoring_incomefilled['age'] > 18]

In [29]:
credit_scoring_incomefilled['age'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

### Afterthoughts: 

1. by taking a look in the age column there were unrealistic values like '0' and '2' and as these are less than 1% of the whole data so I decided to drop these rows.


## Education: 

In [30]:
credit_scoring_incomefilled['education'].value_counts()

secondary education    13640
bachelor's degree       4674
SECONDARY EDUCATION      768
Secondary Education      702
some college             664
BACHELOR'S DEGREE        271
Bachelor's Degree        266
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

In [31]:
credit_scoring_incomefilled['education_id'].value_counts()

1    15110
0     5211
2      740
3      282
4        6
Name: education_id, dtype: int64

In [32]:
credit_scoring_incomefilled['education'] = credit_scoring_incomefilled['education'].str.lower()

In [33]:
credit_scoring_incomefilled['education'].value_counts()

secondary education    15110
bachelor's degree       5211
some college             740
primary education        282
graduate degree            6
Name: education, dtype: int64

### Afterthoughts: 

1. by comparing the unique values and there count between 'education' and 'education_id' we notice that there wern't identicatl.


2. by looking in the unique values of the 'education' column we find that there are different cases for the same value so to unify it I changed the strings to lower case and that made the values_counts between 'education' and 'education_id'identical.


## Family Status: 

In [34]:
credit_scoring_incomefilled['family_status'].value_counts()

married              12283
civil partnership     4144
unmarried             2788
divorced              1183
widow / widower        951
Name: family_status, dtype: int64

In [35]:
credit_scoring_incomefilled['family_status_id'].value_counts()

0    12283
1     4144
4     2788
3     1183
2      951
Name: family_status_id, dtype: int64

### Afterthoughts: 

1. here we don't need to change anything as there are no duplicate values with different cases and the value_counts are identical between 'family_status' and 'family_status_id'.

## Gender: 

In [36]:
credit_scoring_incomefilled['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [37]:
credit_scoring_incomefilled['gender'].value_counts()

F      14118
M       7230
XNA        1
Name: gender, dtype: int64

In [38]:
credit_scoring_incomefilled.loc[credit_scoring_incomefilled['gender'] == 'M','gender'] = credit_scoring_incomefilled.loc[credit_scoring_incomefilled['gender'] == 'M'] = 'Male'

In [39]:
credit_scoring_incomefilled.loc[credit_scoring_incomefilled['gender'] == 'F','gender'] = credit_scoring_incomefilled.loc[credit_scoring_incomefilled['gender'] == 'F'] = 'Female'

In [40]:
credit_scoring_incomefilled = credit_scoring_incomefilled.loc[credit_scoring_incomefilled['gender'] != 'XNA']
credit_scoring_incomefilled['gender'].value_counts()

Female    14118
Male       7230
Name: gender, dtype: int64

In [41]:
credit_scoring_incomefilled

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,Female,employee,0,40620.1,purchase of the house
1,1,4024,36,secondary education,1,married,0,Female,employee,0,17932.8,car purchase
2,0,5623,33,secondary education,1,married,0,Male,employee,0,23341.8,purchase of the house
3,3,4124,32,secondary education,1,married,0,Male,employee,0,42820.6,supplementary education
4,0,340266,53,secondary education,1,civil partnership,1,Female,retiree,0,25378.6,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,secondary education,1,civil partnership,1,Female,business,0,35966.7,housing transactions
21521,0,343937,67,secondary education,1,married,0,Female,retiree,0,24960,purchase of a car
21522,1,2113,38,secondary education,1,civil partnership,1,Male,employee,1,14347.6,property
21523,3,3112,38,secondary education,1,married,0,Male,employee,1,39054.9,buying my own car


### Afterthoughts: 

1. we notice the weird value in gender 'XNA' and as there is only one value of this kind and no way to figure out what this means so I dropped it.


2. I changed the vlaues from F,M to Female and Male to mnake it clearer.

## Income Type: 

In [42]:
credit_scoring_incomefilled['income_type'].value_counts()

employee                       11022
business                        5042
retiree                         3827
civil servant                   1451
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

### Afterthoughts: 

1. here we don't need to change anything as there are no duplicate values with different cases.

## Debt: 

In [43]:
credit_scoring_incomefilled['debt'].value_counts()

0    19623
1     1725
Name: debt, dtype: int64

In [44]:
credit_scoring_incomefilled = credit_scoring_incomefilled.rename(columns={'debt':'defaulted'})  # rename the column

In [45]:
credit_scoring_incomefilled['defaulted'] = credit_scoring_incomefilled['defaulted'].astype('bool')
credit_scoring_incomefilled['defaulted'].value_counts()

False    19623
True      1725
Name: defaulted, dtype: int64

### Afterthoughts: 

1. to make everything clearer I changed the name of debt column to defaulted and chaged the type to boolean as the results could only be either yes or no.

## Total income: 

In [46]:
credit_scoring_incomefilled['total_income'] = credit_scoring_incomefilled['total_income'].astype('int')
credit_scoring_incomefilled['total_income'].describe()

count     21348.000000
mean      26792.576588
std       15650.027542
min        3306.000000
25%       17246.750000
50%       25021.500000
75%       31274.750000
max      362496.000000
Name: total_income, dtype: float64

### Afterthoughts: 

1. I changed the total_income type from float to int just to make it clearer but either ways it is correct.


2. in the income there are no negative values so we don't need to change anything else but we need to remember that some of the values were missing and were filled with the mean.

## Purpose: 

In [47]:
credit_scoring_incomefilled['purpose'].value_counts()

wedding ceremony                            791
having a wedding                            769
to have a wedding                           765
real estate transactions                    672
buy commercial real estate                  658
buying property for renting out             650
transactions with commercial real estate    645
housing transactions                        643
purchase of the house for my family         639
purchase of the house                       638
housing                                     637
property                                    629
transactions with my real estate            629
construction of own property                627
building a real estate                      622
building a property                         620
purchase of my own house                    619
buy real estate                             618
housing renovation                          608
buy residential real estate                 602
buying my own car                       

### Afterthoughts: 

1. we can notice that there are serveral values with the same meaning but different wording so this will need further work with stemming and lemmatization to unify the purposes. in my opinion this needs to be done after finding the duplicates to avoid creating more and unnecessary duplicates.

# Duplicates:

In [48]:
duplicates = credit_scoring_incomefilled.duplicated().sum()
duplicates

71

In [49]:
duplicates_Percentage = (duplicates / len(credit_scoring_incomefilled)) * 100
duplicates_Percentage

0.3325838486040847

In [50]:
credit_scoring_incomefilled = credit_scoring_incomefilled.drop_duplicates().reset_index(drop = True)
credit_scoring_incomefilled.duplicated().sum()

0

### Notes:

1. There are 71 duplicated rows which make around 0.3 % of the data and that is fairly small and will not make an impcat on the results so I dropped them. now our data have no missing values and duplicates so we continue to trhe next step which is Categorizing the data.

# Categorizing Data:

## Purpose:

<div class="alert alert-success" role="alert">
Reviewer's comment v. 2:
    
Yes, drop these duplicates.
</div>

In [51]:
def lemmatize_count(purpose):
    token = nltk.word_tokenize(purpose)
    lemma = [wordnet_lemma.lemmatize(w, pos = 'n') for w in token]
    return lemma
lemma_result = credit_scoring['purpose'].apply(lemmatize_count)
lemma_result

0        [purchase, of, the, house]
1                   [car, purchase]
2        [purchase, of, the, house]
3        [supplementary, education]
4            [to, have, a, wedding]
                    ...            
21520        [housing, transaction]
21521        [purchase, of, a, car]
21522                    [property]
21523        [buying, my, own, car]
21524             [to, buy, a, car]
Name: purpose, Length: 21525, dtype: object

In [52]:
count_lemma_result = pd.Series(Counter([W for L in lemma_result for W in L]))
count_lemma_result

purchase         3314
of               2998
the              1288
house            1908
car              4315
supplementary     909
education        3114
to               3081
have              774
a                5130
wedding          2348
housing          1912
transaction      2610
having            777
for              1294
my               2396
family            641
buy              2367
real             4478
estate           4478
commercial       1315
residential       607
construction      635
own              2240
property         2542
building         1246
buying           1637
second-hand       968
with             1281
become            412
educated          412
getting           869
an                443
ceremony          797
get               447
higher            426
profile           436
university        949
renting           653
out               653
renovation        612
going             496
dtype: int64

In [53]:
def purpose_cat(lemma):
    categories = {'education': ['education', 'educated','university'],
                'real estate': ['house', 'housing', 'estate','property',],
                'car': ['car'],
                'wedding': ['wedding', 'ceremony']}
    for word in lemma:
        for cat in categories:
            if word in categories[cat]:
                return cat

#def lemmatize(purpose):
   # token = nltk.word_tokenize(purpose)
   #lemma = [wordnet_lemma.lemmatize(w, pos = 'n') for w in token]
   # return purpose_cat(lemma)

In [54]:
credit_scoring_incomefilled['purpose_category'] = lemma_result.apply(purpose_cat)
credit_scoring_incomefilled

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,defaulted,total_income,purpose,purpose_category
0,1,8437,42,bachelor's degree,0,married,0,Female,employee,False,40620,purchase of the house,real estate
1,1,4024,36,secondary education,1,married,0,Female,employee,False,17932,car purchase,car
2,0,5623,33,secondary education,1,married,0,Male,employee,False,23341,purchase of the house,real estate
3,3,4124,32,secondary education,1,married,0,Male,employee,False,42820,supplementary education,education
4,0,340266,53,secondary education,1,civil partnership,1,Female,retiree,False,25378,to have a wedding,wedding
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21272,1,4529,43,secondary education,1,civil partnership,1,Female,business,False,35966,housing transactions,real estate
21273,0,343937,67,secondary education,1,married,0,Female,retiree,False,24959,purchase of a car,real estate
21274,1,2113,38,secondary education,1,civil partnership,1,Male,employee,True,14347,property,wedding
21275,3,3112,38,secondary education,1,married,0,Male,employee,True,39054,buying my own car,real estate


#### Afterthoughts:

1. we used lemmatization to find a list of the used words and from these words i applied a function to split the purpose into 4 categories: Eduaction - Wedding - Real Estate - Car

## Total Income:

In [55]:
credit_scoring_incomefilled['total_income'].describe()

count     21277.000000
mean      26792.595197
std       15676.118592
min        3306.000000
25%       17219.000000
50%       24966.000000
75%       31320.000000
max      362496.000000
Name: total_income, dtype: float64

In [56]:
def total_income_cat(income):
    if income < 17000:
        income_cat = 'low income'
    elif income < 25000:
        income_cat = 'below average income'
    elif income < 32000:
        income_cat = 'above average income'
    else:
        income_cat = 'high income'
    return income_cat

In [57]:
credit_scoring_incomefilled['total_income_category'] = credit_scoring_incomefilled['total_income'].apply(total_income_cat)
credit_scoring_incomefilled

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,defaulted,total_income,purpose,purpose_category,total_income_category
0,1,8437,42,bachelor's degree,0,married,0,Female,employee,False,40620,purchase of the house,real estate,high income
1,1,4024,36,secondary education,1,married,0,Female,employee,False,17932,car purchase,car,below average income
2,0,5623,33,secondary education,1,married,0,Male,employee,False,23341,purchase of the house,real estate,below average income
3,3,4124,32,secondary education,1,married,0,Male,employee,False,42820,supplementary education,education,high income
4,0,340266,53,secondary education,1,civil partnership,1,Female,retiree,False,25378,to have a wedding,wedding,above average income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21272,1,4529,43,secondary education,1,civil partnership,1,Female,business,False,35966,housing transactions,real estate,high income
21273,0,343937,67,secondary education,1,married,0,Female,retiree,False,24959,purchase of a car,real estate,below average income
21274,1,2113,38,secondary education,1,civil partnership,1,Male,employee,True,14347,property,wedding,low income
21275,3,3112,38,secondary education,1,married,0,Male,employee,True,39054,buying my own car,real estate,high income


#### Afterthoughts:

1. the total income was categorized based on the results of the describe method and were split into 4 categories: low income - below average income - above average income - hight income

* **for now these are the obvious categories thatt we need but we may add other categories as needed.**

# Answer these questions:

### 1. Is there a relation between having kids and repaying a loan on time?

In [58]:
pivot_children = credit_scoring_incomefilled.pivot_table(index ='children',columns ='defaulted',values='gender',aggfunc='count')
pivot_children['default-rate'] = (pivot_children[True] / (pivot_children[False] + pivot_children[True]))*100 
pivot_children

defaulted,False,True,default-rate
children,,,
0,12963.0,1058.0,7.545824
1,4397.0,442.0,9.134119
2,1845.0,194.0,9.514468
3,301.0,27.0,8.231707
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


In [59]:
pivot_children_gender = credit_scoring_incomefilled.pivot_table(index =['children','gender'], columns ='defaulted',values='education_id',aggfunc='count')
pivot_children_gender['default-rate'] = (pivot_children_gender[True] / (pivot_children_gender[False] + pivot_children_gender[True]))*100 
pivot_children_gender.head(15)

defaulted         False   True  default-rate
children gender                             
0        Female  8881.0  589.0      6.219641
         Male    4082.0  469.0     10.305427
1        Female  2865.0  244.0      7.848183
         Male    1532.0  198.0     11.445087
2        Female  1113.0  134.0     10.745790
         Male     732.0   60.0      7.575758
3        Female   178.0   17.0      8.717949
         Male     123.0   10.0      7.518797
4        Female    27.0    1.0      3.571429
         Male      10.0    3.0     23.076923
5        Female     7.0    NaN           NaN
         Male       2.0    NaN           NaN

In [60]:
pivot_children_family = credit_scoring_incomefilled.pivot_table(index =['children','family_status'], columns ='defaulted',values='education_id',aggfunc='count')
pivot_children_family['default-rate'] = (pivot_children_family[True] / (pivot_children_family[False] + pivot_children_family[True]))*100 
pivot_children_family

defaulted                    False   True  default-rate
children family_status                                 
0        civil partnership  2489.0  227.0      8.357879
         divorced            722.0   55.0      7.078507
         married            6923.0  514.0      6.911389
         unmarried          2039.0  210.0      9.337483
         widow / widower     790.0   52.0      6.175772
1        civil partnership   876.0  118.0     11.871227
         divorced            294.0   21.0      6.666667
         married            2752.0  245.0      8.174842
         unmarried           401.0   51.0     11.283186
         widow / widower      74.0    7.0      8.641975
2        civil partnership   311.0   30.0      8.797654
         divorced             72.0    7.0      8.860759
         married            1380.0  145.0      9.508197
         unmarried            65.0    9.0     12.162162
         widow / widower      17.0    3.0     15.000000
3        civil partnership    48.0    8.0     14.285714
         divorced             10.0    1.0      9.090909
         married             230.0   17.0      6.882591
         unmarried             7.0    1.0     12.500000
         widow / widower       6.0    NaN           NaN
4        civil partnership     8.0    NaN           NaN
         divorced              1.0    NaN           NaN
         married              26.0    3.0     10.344828
         unmarried             1.0    1.0     50.000000
         widow / widower       1.0    NaN           NaN
5        civil partnership     2.0    NaN           NaN
         married               7.0    NaN           NaN

In [61]:
def children_cat(child):
    if child == 0:
        cat = 'no children'
    elif child <= 2:
        cat = '2 or less'
    else:
        cat = '3 or more'
    return cat
credit_scoring_incomefilled['children_category']=credit_scoring_incomefilled['children'].apply(children_cat)
credit_scoring_incomefilled['children_category'].value_counts()

no children    14021
2 or less       6878
3 or more        378
Name: children_category, dtype: int64

In [62]:
pivot_children_cat_gender = credit_scoring_incomefilled.pivot_table(index =['children_category', 'gender'],columns ='defaulted',values='education_id',aggfunc='count')
pivot_children_cat_gender
pivot_children_cat_gender['default-rate'] = (pivot_children_cat_gender[True] / (pivot_children_cat_gender[False] + pivot_children_cat_gender[True]))*100 
pivot_children_cat_gender

defaulted                 False  True  default-rate
children_category gender                           
2 or less         Female   3978   378      8.677686
                  Male     2264   258     10.229976
3 or more         Female    212    18      7.826087
                  Male      135    13      8.783784
no children       Female   8881   589      6.219641
                  Male     4082   469     10.305427

In [63]:
pivot_children_cat = credit_scoring_incomefilled.pivot_table(index ='children_category',columns ='defaulted',values='education_id',aggfunc='count')
pivot_children_cat
pivot_children_cat['default-rate'] = (pivot_children_cat[True] / (pivot_children_cat[False] + pivot_children_cat[True]))*100 
pivot_children_cat

defaulted,False,True,default-rate
children_category,,,
2 or less,6242,636,9.246874
3 or more,347,31,8.201058
no children,12963,1058,7.545824


### Conclusion: 

* as we can find from the table above the default rate varries between 7.5 - 9.7 %.
* the customers with no children have 7.5% chance of defaulting on aloan then it increases and peaks around 19.7 % with 4 kids. 
* after splitting the customers to groups based on kids numbers the difference will decrease and the default rate will be between 7.5 - 9.2%
* when taking the gender into consideration we will find the results are a bit different:
1. Women: the default rate decreases between women with no children 6.2 % and increases then peaks with women with 2 kids 10.7%
2. Men: the default rate will be higher between men with no children or just one child (10.5 - 11.4 %) and it decreases for men with 2 or 3 kids (7.5 %). 


### 2. Is there a relation between marital status and repaying a loan on time?

In [64]:
pivot_marital = credit_scoring_incomefilled.pivot_table(index ='family_status',columns ='defaulted',values='education_id',aggfunc='count')
pivot_marital['default-rate'] = (pivot_marital[True] / (pivot_marital[False] + pivot_marital[True]))*100 
pivot_marital

defaulted,False,True,default-rate
family_status,,,
civil partnership,3734,383,9.302890
divorced,1099,84,7.100592
married,11318,924,7.547786
unmarried,2513,272,9.766607
widow / widower,888,62,6.526316


### Conclusion: 

* as we can find from the table above the default rate varries between 6.5 - 9.7%.
* the chance that the customers will default on a loan increases if the the costumers are either unmarried or in a civil partnership. (9.7 - 9.3%)
* the widowers are the least likely to default on a loan with a default rate of 6.5 %
* married and divorced customers have relatively the same chances of defaulting on a loan which is between 7.1 and 7.5%


### 3. Is there a relation between income level and repaying a loan on time?

In [65]:
pivot_income = credit_scoring_incomefilled.pivot_table(index ='total_income_category',columns ='defaulted',values='education_id',aggfunc='count')
pivot_income['default-rate'] = (pivot_income[True] / (pivot_income[False] + pivot_income[True]))*100 
pivot_income

defaulted,False,True,default-rate
total_income_category,,,
above average income,5115,483,8.628081
below average income,5006,479,8.732908
high income,4664,355,7.073122
low income,4767,408,7.884058


In [66]:
pivot_income_type = credit_scoring_incomefilled.pivot_table(index =['total_income_category', 'income_type'],columns ='defaulted',values='education_id',aggfunc='count')
pivot_income_type['default-rate'] = (pivot_income_type[True] / (pivot_income_type[False] + pivot_income_type[True]))*100 
pivot_income_type

defaulted                                           False   True  default-rate
total_income_category income_type                                             
above average income  business                     1322.0  110.0      7.681564
                      civil servant                 369.0   22.0      5.626598
                      employee                     2604.0  289.0      9.989630
                      entrepreneur                    1.0    NaN           NaN
                      retiree                       819.0   62.0      7.037457
below average income  business                     1054.0   95.0      8.268059
                      civil servant                 329.0   23.0      6.534091
                      employee                     2723.0  306.0     10.102344
                      retiree                       900.0   55.0      5.759162
high income           business                     1630.0  111.0      6.375646
                      civil servant                 349.0   18.0      4.904632
                      employee                     2167.0  195.0      8.255715
                      entrepreneur                    1.0    NaN           NaN
                      retiree                       516.0   31.0      5.667276
                      unemployed                      1.0    NaN           NaN
low income            business                      656.0   57.0      7.994390
                      civil servant                 316.0   23.0      6.784661
                      employee                     2444.0  259.0      9.581946
                      paternity / maternity leave     NaN    1.0           NaN
                      retiree                      1350.0   67.0      4.728299
                      student                         1.0    NaN           NaN
                      unemployed                      NaN    1.0           NaN

### Conclusion: 

* as we can find from the table above the default rate varries between 7.0 - 8.7 % 
* the chance that the customers will default on a loan is highest with the customers with average income.
* surprisingly the custumers with low income are not the ones with the highest defaulting chance as they have a chance of 7.8%
* unsurprisingly the customers with high income are the least likely to default on a loan but they are not that far as they have 7.0% chance.
* when taking the income type into consideration we find that the employees have the highest default rate throughout the income level groups.
* civil servants and retiree have the lowest default rate.


### 4. How do different loan purposes affect on-time repayment of the loan?

In [67]:
pivot_purpose = credit_scoring_incomefilled.pivot_table(index ='purpose_category',columns ='defaulted',values='education_id',aggfunc='count')
pivot_purpose['default-rate'] = (pivot_purpose[True] / (pivot_purpose[False] + pivot_purpose[True]))*100 
pivot_purpose

defaulted,False,True,default-rate
purpose_category,,,
car,3923,341,7.997186
education,3666,321,8.051166
real estate,9838,869,8.116186
wedding,2125,194,8.365675


### Conclusion: 

* as we can find from the table above default rate varries between 8.0 - 8.3 %.
* the difference in the chance to default on a loan between customers with different purposes is not that high (around 0.3 %).
* the customers with the highest chance to default on a loan are the ones who need the loan for a wedding and the default rate is lowest for people who wants to buy a car.

## Additional observation

In [68]:
pivot_education = credit_scoring_incomefilled.pivot_table(index = 'education',columns ='defaulted',values='education_id',aggfunc='count')
pivot_education['default-rate'] = (pivot_education[True] / (pivot_education[False] + pivot_education[True]))*100 
pivot_education

defaulted,False,True,default-rate
education,,,
bachelor's degree,4924.0,277.0,5.325899
graduate degree,6.0,NaN,NaN
primary education,251.0,31.0,10.992908
secondary education,13700.0,1349.0,8.964051
some college,671.0,68.0,9.201624


In [69]:
credit_scoring_incomefilled.groupby('education')['total_income'].mean()

education
bachelor's degree      32536.338781
graduate degree        27959.500000
primary education      21564.563830
secondary education    24804.101203
some college           28847.993234
Name: total_income, dtype: float64

### Conclusion: 

* as we can find from the table above default rate varries between 5.3 - 10.9 %.
* we can notice that the lower the education level  the higher the default rate with an exception for people who have attended some college which i can consider vague and these have relatively the same rate as the customers who have just finished secondary education so they have almost the same level of education.
* noticably the higher the education the higher the income average.

# Final Conclusion:

in the analysis we took into consideration the following points:
* number of kids
* marital status
* income level
* purpose of the loan
* education level

and after looking at the resuls I came to the conclusion that the follwing attributes were found to correlate with higher default rates:

* males with less than 2 kids.
* females with more than 1 kid.
* in general( customer with 1-2 kids have the highest default rate)
* Being unmarried or in a civil partnership
* Being an employee
* Taking out a loan for a wedding
* having a low level of education 


* **as for the income level there was no relation between the level and defaulting so it should not be taken into consideration.**
* *final note: as the higher education level has a lower default rate, it can be taken into consideration that a loan with the purpose of education will correspond to this and the default rate will decrease but as we have no data regarding if the education was a success or not so we can't elaborate more on this point.*